In [ ]:
pip install gdown

In [ ]:
import os

In [ ]:
def download_files():
  en_hi_data_link = "https://drive.google.com/file/d/15Nmz2hTtDv8V8JTOM0ysstZ1vXWugKbp/view?usp=sharing"
  en_hi_model_data_link = "https://drive.google.com/file/d/1BzxPJu8nVFLML9Z_jM6K8Hg98mEXeehg/view?usp=sharing"
  gdrive_url = "https://drive.google.com/uc?id="

  data_id = en_hi_data_link.split("/d")[2].split("/view")[0].strip("/")
  model_id = en_hi_model_data_link.split("/d")[2].split("/view")[0].strip("/")

  os.environ["url1"] = gdrive_url + data_id
  os.environ["url2"] = gdrive_url + model_id

  !gdown $url1
  !gdown $url2


### Please run download_files() to fetch necessary data and model check point to setup the NMT

In [ ]:
download_files()

Downloading...
From: https://drive.google.com/uc?id=15Nmz2hTtDv8V8JTOM0ysstZ1vXWugKbp
To: /content/en_hi_domain_adapation_data.zip
100% 935k/935k [00:00<00:00, 57.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1BzxPJu8nVFLML9Z_jM6K8Hg98mEXeehg
To: /content/en_hi_checkpoint.zip
589MB [00:03, 182MB/s]


# Installing JOEY NMT

In [ ]:
# Install JoeyNMT
! git clone https://github.com/may-/joeynmt.git
! cd joeynmt; pip3 install .
# Install Pytorch with GPU support v1.9.0
! pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

Cloning into 'joeynmt'...
remote: Enumerating objects: 3426, done.
remote: Counting objects: 100% (475/475), done.
remote: Compressing objects: 100% (227/227), done.
remote: Total 3426 (delta 296), reused 375 (delta 248), pack-reused 2951
Receiving objects: 100% (3426/3426), 12.57 MiB | 16.98 MiB/s, done.
Resolving deltas: 100% (2324/2324), done.
Processing /content/joeynmt
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 90 kB 332 kB/s 
     |████████████████████████████████| 636 kB 19.6 MB/s 
     |████████████████████████████████| 392 kB 53.4 MB/s 
     |████████████████████████████████| 106 kB 54.0 MB/

In [ ]:
!pip install sacremoses

## clone indic nlp resources repo
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
## install indic nlp library
!pip install indic-nlp-library
## export indic nlp resource library
import os
os.environ["INDIC_RESOURCES_PATH"] = "/content/indic_nlp_resources"

     |████████████████████████████████| 895 kB 7.9 MB/s 
Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 139 (delta 2), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (139/139), 149.77 MiB | 29.63 MiB/s, done.
Resolving deltas: 100% (53/53), done.
Checking out files: 100% (28/28), done.
     |████████████████████████████████| 40 kB 3.8 MB/s 
     |████████████████████████████████| 2.8 MB 17.9 MB/s 


In [ ]:
!rm -rf /content/joeynmt/data/enhi/*
!rm -rf /content/joeynmt/models/enhi/*

!mkdir -p /content/joeynmt/data/enhi
!mkdir -p /content/joeynmt/models/enhi

!unzip en_hi_domain_adapation_data.zip -d /content/joeynmt/data/enhi
!unzip en_hi_checkpoint.zip  -d /content/joeynmt/models/enhi

Archive:  en_hi_domain_adapation_data.zip
  inflating: /content/joeynmt/data/enhi/dev.bpe.en  
  inflating: /content/joeynmt/data/enhi/dev.bpe.hi  
  inflating: /content/joeynmt/data/enhi/test.bpe.en  
  inflating: /content/joeynmt/data/enhi/test.bpe.hi  
  inflating: /content/joeynmt/data/enhi/train.bpe.en  
  inflating: /content/joeynmt/data/enhi/train.bpe.hi  
  inflating: /content/joeynmt/data/enhi/vocab.txt  
  inflating: /content/joeynmt/data/enhi/len_full.bpe.codes.5000  
  inflating: /content/joeynmt/data/enhi/vocab.en  
Archive:  en_hi_checkpoint.zip
  inflating: /content/joeynmt/models/enhi/69100.hyps  
  inflating: /content/joeynmt/models/enhi/69200.hyps  
  inflating: /content/joeynmt/models/enhi/69300.hyps  
  inflating: /content/joeynmt/models/enhi/69400.hyps  
  inflating: /content/joeynmt/models/enhi/69500.hyps  
  inflating: /content/joeynmt/models/enhi/69600.hyps  
  inflating: /content/joeynmt/models/enhi/69700.hyps  
  inflating: /content/joeynmt/models/enhi/69800.h

In [ ]:
def create_model_config():
  model_type = "bpe"
  use_cuda = "true"
  config = """
  name: "{name}"

  data:
      src: "{src_lang}"
      trg: "{tgt_lang}"
      train: "data/{name}/train.{model_type}"
      dev:   "data/{name}/dev.{model_type}"
      test:   "data/{name}/test.{model_type}"
      level: "{model_type}"
      lowercase: False
      max_sent_length: 60
      src_vocab: "data/{name}/vocab.txt"
      trg_vocab: "data/{name}/vocab.txt"

  testing:
      beam_size: 5
      alpha: 1.0

  training:
      load_model: "models/{name}/best.ckpt"
      random_seed: 42
      optimizer: "adam"
      normalization: "tokens"
      adam_betas: [0.9, 0.999] 
      scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
      patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
      learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
      learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
      decrease_factor: 0.7
      loss: "crossentropy"
      learning_rate: 0.0003
      learning_rate_min: 0.00000001
      weight_decay: 0.0
      label_smoothing: 0.1
      batch_size: 4096
      batch_type: "token"
      eval_batch_size: 3600
      eval_batch_type: "token"
      batch_multiplier: 1
      early_stopping_metric: "ppl"
      epochs: 100                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
      validation_freq: 1000          # TODO: Set to at least once per epoch.
      logging_freq: 100
      eval_metric: "bleu"
      model_dir: "models/{name}"
      overwrite: True               # TODO: Set to True if you want to overwrite possibly existing models. 
      shuffle: True
      use_cuda: {use_cuda}
      max_output_length: 130
      print_valid_sents: [0, 1, 2, 3]
      keep_best_ckpts: 3

  model:
      initializer: "xavier"
      bias_initializer: "zeros"
      init_gain: 1.0
      embed_initializer: "xavier"
      embed_init_gain: 1.0
      tied_embeddings: True
      tied_softmax: True
      encoder:
          type: "transformer"
          num_layers: 6
          num_heads: 4             # TODO: Increase to 8 for larger data.
          embeddings:
              embedding_dim: 256   # TODO: Increase to 512 for larger data.
              scale: True
              dropout: 0.2
          # typically ff_size = 4 x hidden_size
          hidden_size: 256         # TODO: Increase to 512 for larger data.
          ff_size: 1024            # TODO: Increase to 2048 for larger data.
          dropout: 0.3
      decoder:
          type: "transformer"
          num_layers: 6
          num_heads: 4              # TODO: Increase to 8 for larger data.
          embeddings:
              embedding_dim: 256    # TODO: Increase to 512 for larger data.
              scale: True
              dropout: 0.2
          # typically ff_size = 4 x hidden_size
          hidden_size: 256         # TODO: Increase to 512 for larger data.
          ff_size: 1024            # TODO: Increase to 2048 for larger data.
          dropout: 0.3
  """.format(name = "enhi", src_lang = "en", tgt_lang = "hi", model_type = model_type, use_cuda = use_cuda)
  with open("joeynmt/configs/{name}.yaml".format(name = "enhi"), 'w') as f:
      f.write(config)

### Please run create_model_config() to create the Joey NMT configuration for NMT

In [ ]:
create_model_config()

In [ ]:
def translate_file_en_hi(input_file):
  import os
  from indicnlp.tokenize import indic_detokenize

  config = "/content/joeynmt/configs/enhi.yaml"
  os.environ["bpe_file"] = "len_full.bpe.codes.5000"
  os.environ["src_lang"] = "en"
  os.environ["config"] = config


  !sacremoses tokenize < "$input_file" > input_tokenized.txt
  !subword-nmt apply-bpe -c /content/joeynmt/data/enhi/$bpe_file --vocabulary /content/joeynmt/data/enhi/vocab.$src_lang < input_tokenized.txt > joeynmt/translate.txt
  !cd joeynmt; python3 -m joeynmt translate $config < translate.txt > joey_pred.txt
  print()

  with open(input_file, 'r') as inputtext, open('joeynmt/joey_pred.txt', 'r') as trtext:
    input_lines = inputtext.readlines()
    translated_lines = trtext.readlines()
    for line1,line2 in zip(input_lines, translated_lines):
      print("Source Line: ", line1)
      print("Translated Line: ", indic_detokenize.trivial_detokenize(line2,lang='hi'))
      print()

Below given lines are samples taken from testing data set. You can also update the "input_data" variable to create a new file with updated test sentences

In [ ]:
input_data = """It can cause a new continuous cough, fever or loss of, or change in, sense of smell or taste (anosmia).
Generally, corona virus can cause more severe symptoms in people with weakened immune systems, older people and those with long term conditions like diabetes, cancer and chronic lung disease.
This is a rapidly changing situation which is being monitored carefully.
What you can and cannot do depends on your protection level (tier).
Glasgow and Moray are at level 3.
Most of mainland Scotland and some islands are at level 2.
Some islands are at level 1.
Check the protection level of your area
Read the Scottish Governments guidance on protection levels
Get a coronavirus test
Some people are at higher risk of developing severe illness with corona virus.
These people should strictly follow physical distancing measures.
Their household and other contacts should also strictly follow physical distancing advice.
"""

with open('input.txt', 'w') as wf:
  wf.write(input_data)

### Please run translate_file_in_en() providing "input.txt" as argument to the function

In [ ]:
translate_file_en_hi('input.txt')

2021-09-24 02:58:36,365 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-09-24 02:58:36,962 - INFO - joeynmt.prediction - Loading model from models/enhi/latest.ckpt
2021-09-24 02:58:37,186 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-09-24 02:58:37,623 - INFO - joeynmt.model - Enc-dec model built.

Source Line:  It can cause a new continuous cough, fever or loss of, or change in, sense of smell or taste (anosmia).

Translated Line:  यह एक नयी निरंतर खाँसी, ज्वर या कमी उत्पन्न कर सकता है, या मस्तिष्क या तनाव की संवेदनशीलता में परिवर्तन कर सकता है।


Source Line:  Generally, corona virus can cause more severe symptoms in people with weakened immune systems, older people and those with long term conditions like diabetes, cancer and chronic lung disease.

Translated Line:  सामान्यतः, कोरोनावायरस कमजोर प्रतिरक्षी तंत्र वाले लोगों में अधिक गंभीर लक्षण उत्पन्न कर सकता है और वे मधुमेह, कैंसर और चिरकालिक रोग जैसी दीर्घकालिक स्थितियों से उत्पन्न कर सकते हैं।


So